In [2]:
import numpy as np
data=np.load("data_small_100.npy")
target=np.load("target_small_100.npy")

In [3]:
data.shape

(100, 224, 224, 3)

In [4]:
target.shape

(100, 100)

In [5]:
#transfer learning

from keras.models import Model,Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import preprocess_input,VGG16

IMAGE_SIZE = [224, 224]
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg.layers:
  layer.trainable = False

x = Flatten()(vgg.output)
prediction = Dense(target.shape[1], activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


Init Plugin
Init Graph Optimizer
Init Kernel
Metal device set to: Apple M1


2023-02-21 22:53:03.779140: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-21 22:53:03.779256: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
#our own CNN 
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Conv2D,MaxPooling2D,Dropout,BatchNormalization

model=Sequential()

model.add(Conv2D(256,(3,3),input_shape=data.shape[1:],kernel_initializer="orthogonal"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3),kernel_initializer="orthogonal"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(1000,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(128,activation="relu"))
model.add(Dense(target.shape[1],activation="softmax"))

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

model.summary()


          

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 256)     7168      
_________________________________________________________________
batch_normalization (BatchNo (None, 222, 222, 256)     1024      
_________________________________________________________________
activation (Activation)      (None, 222, 222, 256)     0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 256)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 128)     295040    
_________________________________________________________________
batch_normalization_1 (Batch (None, 109, 109, 128)     512       
_________________________________________________________________
activation_1 (Activation)    (None, 109, 109, 128)     0

In [7]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)


In [8]:
from keras.callbacks import ModelCheckpoint

checkpoint=ModelCheckpoint("model/model-{epoch:03d}.h5",monitor="val _loss",save_best_only=True,mode="auto")

In [9]:
#augmentation

from keras.preprocessing.image import ImageDataGenerator

aug=ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,
                      horizontal_flip=True,fill_mode="nearest")



In [ ]:
bs=32

history=model.fit(aug.flow(train_data,train_target,batch_size=bs)
                  ,steps_per_epoch=len(train_data)//bs
                  ,validation_data=(test_data,test_target)
                  ,validation_steps=len(test_data)//bs
                  ,epochs=2)

2023-02-21 22:53:09.712376: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-02-21 22:53:09.713758: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/2


2023-02-21 22:53:17.654257: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 45s 20s/step - loss: 6.0224 - accuracy: 0.0000e+00
Epoch 2/2


In [1]:
model.save_weights("vgg16_face.h5")

NameError: name 'model' is not defined

In [ ]:
model.save("vgg16_face.model")